In [1]:
%pylab qt5
import matplotlib.pylab as plt
import pandas as pd
import numpy as np
import time
import glob #to import several files within a folder
# import scipy.signal
import ecgprocpy3
# from sklearn import datasets, linear_model
# from scipy import stats
# from scipy import signal
import liblo
import pyson

Populating the interactive namespace from numpy and matplotlib


# Import data

In [2]:
directory='ecgdata/Bicycledata'
files = glob.glob(directory+"/*.csv")
bikedata = []
for f in files:
    bikedata.append(pd.read_csv(f, delimiter=' ', header = None,  usecols=[0,1]))
for dataset in range(len(bikedata)):
    bikedata[dataset].columns = ["time","L1"]

# Data exploration

In [13]:
data = bikedata[0]["L1"]; sr = 1000.0; window = 400; 
t = np.linspace(0, len(data)/sr, len(data))
plt.plot(t, data)
plt.title('ECG data')
plt.ylabel('Amplitude [mV]')
plt.xlabel('Time [s]')

Text(0.5, 0, 'Time [s]')

In [41]:
peaks= ecgprocpy3.find_R_peaks_onechannel(data, sr, window, thpercentage = 0.4, plot = True)
t_peaks= [x[0] for x in peaks]
rrs = [a - b for a, b in zip(t_peaks[1:], t_peaks[:-1])] 

# OSC functions

In [18]:
#Time difference correction to be used with liblo OSC library:
liblo_time_diff = 2208988800.0
osctarget = liblo.Address("127.0.0.1", 57110)

def sc_bundle(timetag, msgAdr="/s_new", msgArgs=["s1", 2000, 1, 0, "freq", 300, "amp", 0.5]):
    '''Function to send OSC messages as bundle'''
    message = liblo.Message(msgAdr)
    for arg in msgArgs:
        message.add(arg)
    bundle = liblo.Bundle(timetag+liblo_time_diff, message)
    liblo.send(osctarget, bundle)
    
def sc_msg(msgAdr="/s_new", msgArgs=["s1", 2000, 1, 0, "freq", 300, "amp", 0.5]):
    '''Function to send individual OSC messages'''
    message = liblo.Message(msgAdr)
    for arg in msgArgs:
        message.add(arg)
    liblo.send(osctarget, message)   

# SC recording functions using sc3nb

In [19]:
from sc3nb.sc3nb import *

<IPython.core.display.Javascript object>

In [20]:
sc = startup(discard_output=True, verbose=True, liblo_flag=True)   
# scope window should open
%scv s.scope
%scv s.freqscope

[1206/185221.573500:WARNING:resource_bundle_qt.cpp(115)] locale_file_path.empty() for locale 


sc3nb started
-> sc3nb started

-> localhost
sc3> s.scope
-> a Stethoscope

-> a FreqScope



In [ ]:
sc.exit()  # to close sc3

### Define AML synth

In [21]:
%%scv
(
SynthDef("AML", {|amp=1, freq=200, pan=0, out=0, numharm=2, signal|
signal= Blip.ar(freq, numharm, amp)*EnvGen.kr(Env.perc(attackTime: 0.01, releaseTime: 0.2, level: 1, curve: -4), doneAction:2);
Out.ar(out,signal!2);
}).add()
)

-> a SynthDef



In [31]:
#test line synth
now = time.time()
sc.bundle(now, "/s_new", ["AML", -1, 0, 0, 'amp', 0.3, 'freq', 300, 'pan', 0, 'numharm', 2])

# AML sonification real data (and recording) function

In [43]:
def son_aml_rec(signal, sr, playback_rate, window, thpercentage, record = False, recname = 'AML_sonification'):
    
    '''This functions receives an ecg dataset with given parameters and sonifies it
    using the previously defined AML synth. It is also possible to record the sound. 
    
    Parameters:
    signal: array ECG data 
    sr: int/float. sample rate
    window: int. size of windows to search for R peaks in ECG signal
    thpercentage: float [0...1].threshold to find peaks according to a given percentage of amplitude between lowest ECG data value and maximum ECG data value
    record: True/False
    recname: string. filename when sonification is recorded.

    Returns: None
    Generates a sonification and records the audio file to disk if recording parameter is selected.
    '''
    N = len(signal)
    tref = 0.0
    tot_dur = N/sr/playback_rate
    
    peaks= ecgprocpy3.find_R_peaks_onechannel(data, sr, window, thpercentage, plot = False)
    t_peaks= [x[0] for x in peaks]
    now = time.time()

    #init AML variables
    epsilon = 0.05
    delta = 1
          #Recording control
    if record == True:
        filename = 'son01.wav'
        recdir = os.getcwd()+'/sonifications'
        sc.prepare_for_record(0, recdir + recname + filename, 9, 2, "wav", "int16")
        sc.record(now)
    
    for idx, x in enumerate(peaks[:-2]):  
        RRy = peaks[idx+1][0]-peaks[idx][0]
        RRx = peaks[idx+2][0]-peaks[idx+1][0]
        RR = np.abs(RRx - RRy)
        freq = 0.5 * (np.tanh([delta * (RR - epsilon)]) + np.tanh([delta * (RR + epsilon)]))  
        amp = 0.5 * (np.tanh([np.abs(delta * (RR - epsilon))]) + np.tanh([np.abs(delta * (RR + epsilon))]))
        amp_mapped = pyson.linlin(amp, 0, 0.3, pyson.dbamp(-24), pyson.dbamp(-6))
        freq_mapped = pyson.linlin(freq, -1, 1, pyson.midicps(60), pyson.midicps(84))
        sc.bundle(now+t_peaks[idx], "/s_new", ["AML", -1, 0, 0, 'amp', float(amp_mapped), 'freq', float(freq_mapped), 'pan', 0, 'numharm', 2])
    
    if record == True:
        sc.stop_recording(now+tot_dur+0.7)  # to pause, in case you want to resume with record()
    print ('Free All nodes')

## Test the *son_aml_rec* function with real ECG data:

In [44]:
#call AML function:
data = bikedata[0]["L1"][:25000]; sr = 1000.0; window = 400;  
son_aml_rec(data, sr = 1000.0, playback_rate = 1, window = 400, thpercentage = 0.4, record = False)

Free All nodes
